In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
HomeCredit_columns_description = pd.read_csv("../input/home-credit-default-risk/HomeCredit_columns_description.csv",encoding= 'unicode_escape')
POS_CASH_balance = pd.read_csv("../input/home-credit-default-risk/POS_CASH_balance.csv",encoding= 'unicode_escape')
application_test = pd.read_csv("../input/home-credit-default-risk/application_test.csv",encoding= 'unicode_escape')
application_train = pd.read_csv("../input/home-credit-default-risk/application_train.csv",encoding= 'unicode_escape')
bureau = pd.read_csv("../input/home-credit-default-risk/bureau.csv",encoding= 'unicode_escape')
bureau_balance = pd.read_csv("../input/home-credit-default-risk/bureau_balance.csv",encoding= 'unicode_escape')
credit_card_balance = pd.read_csv("../input/home-credit-default-risk/credit_card_balance.csv",encoding= 'unicode_escape')
installments_payments = pd.read_csv("../input/home-credit-default-risk/installments_payments.csv",encoding= 'unicode_escape')
previous_application = pd.read_csv("../input/home-credit-default-risk/previous_application.csv",encoding= 'unicode_escape')
sample_submission = pd.read_csv("../input/home-credit-default-risk/sample_submission.csv",encoding= 'unicode_escape')

#### 상환 가능한 고객이 거부되지 않고 고객이 성공할 수 있도록 원금, 만기 및 상환 일정으로 대출

- application - train / test
  main table, statci Data

  하나의 row가 하나의 대출을 나타낸다.

- bureau

  Credit Bureau에보고 된 다른 금융 기관에서 제공 한 모든 고객의 이전 크레딧

  샘플의 모든 대출에는 신청 날짜 이전에 고객이 신용 관리국에 보유한 크레딧 수만큼의 행이 존재

- bureau_balance
  Credit Bureau에서 이전 크레딧의 월별 잔액

  Bureau에 보고된 모든 이전 크레딧의 월별 이력에 대한 행

- credit_card_balance
  
  신청자가 홈 크레딧으로 보유한 이전 신용 카드의 월별 잔액

- previous_application.csv
  
  대출이있는 고객의 Home Credit 대출에 대한 이전의 모든 신청

- installments_payments
  
  샘플의 대출과 관련하여 Home Credit에서 이전에 지불 된 크레딧에 대한 상환 내역

- HomeCredit_columns_description

- bureau : 
Home Credit에보고 된 다른 금융 기관과의 고객의 이전 대출에 관한 정보. 이전의 각 대출에는 자체 행이 있습니다.
- bureau_balance : 
이전 대출에 대한 월간 정보. 매월 자체 행이 있습니다.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

### Counts of a client's previous loans
- 이전 대출 횟수
- 먼저 다른 금융 기관에서 고객의 이전 대출 수를 가져온다.


- groupby : SK_ID_CURR 열을 기준으로 그룹화
- agg : 열의 평균을 취하는 등 그룹화 된 데이터에 대한 계산을 수행
- merge : SK_ID_CURR 열의 계산 된 통계와 병합 , 해당 값 없는 셀에 Nan값

In [ ]:
bureau.head()

- Credit Bureau에보고 된 다른 금융 기관에서 제공 한 모든 고객의 이전 크레딧

In [ ]:
bureau.info()

In [ ]:
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU' : 'previous_loan_counts'})
previous_loan_counts.head()

In [ ]:
bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].count()

In [ ]:
min(bureau['SK_ID_CURR'])

In [ ]:
train = pd.read_csv("../input/home-credit-default-risk/application_train.csv",encoding= 'unicode_escape')
train = train.merge(previous_loan_counts, on = 'SK_ID_CURR',how='left')

train['previous_loan_counts'] = train['previous_loan_counts'].fillna(0)
train.head()

### Assessing Usefulness of New Variable with r value
새 변수가 유용한 지 확인하기 위해 변수와 대상 간의 Pearson 상관계수 계산

두 변수 사이의 선형관계의 강도를 측정하며 범위는 -1 ~ +1 이다.

변수의 r-값이 클수록 변수의 변경이 대상의 값에 영향을 줄 수 있다.

목표 대비 절대값 r-값이 가장 큰 변수를 찾는다.

#### KDE PLOT - 단일 변수의 분포를 보여준다.

In [ ]:
def kde_target(var_name, df):
    corr = df['TARGET'].corr(df[var_name])
    
    avg_repaid = df.ix[df['TARGET']==0, var_name].median()
    avg_not_repaid = df.ix[df['TARGET']==1, var_name].median()
    
    plt.figure(figsize=(12,6))
    
    sns.kdeplot(df.ix[df['TARGET']==0,var_name],label='TARGET==0')
    sns.kdeplot(df.ix[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    plt.xlabel(var_name)
    plt.ylabel('Density')
    plt.title('%s Distribution' % var_name)
    plt.legend()
    
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

In [ ]:
kde_target('EXT_SOURCE_3', train)

In [ ]:
kde_target('previous_loan_counts', train)

### Numeric Columns 집계

숫자형태의 열에 대한 통계를 계산하기 위해 ID별 그룹화해서 훈련데이터에 다시 병합

In [ ]:
# 클라이언트 id별로 그룹화, 집계 통계 계산
bureau_agg = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
bureau_agg.head()

In [ ]:
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('bureau_%s_%s' % (var, stat))

In [ ]:
#열 이름 목록을 데이터 프레임 열 이름으로
bureau_agg.columns = columns
bureau_agg.head()

In [ ]:
#train데이터와 합병
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
train.head()

In [ ]:
#새로운 상관관계
new_corrs = []


for col in columns:
    corr = train['TARGET'].corr(train[col])
    new_corrs.append((col, corr))

In [ ]:
#상관관계 정렬
new_corrs = sorted(new_corrs, key = lambda x: abs(x[1]), reverse = True)
new_corrs[:15]

새로 만든 변수는 target값과 상관관계가 없음, bureau_DAYS_CREDIT_mean과의 관계 파악

In [ ]:
kde_target('bureau_DAYS_CREDIT_mean', train)

 변수의 평균과 과거에 추가로 대출을 신청 한 고객이 Home Credit에서 대출을 상환 할 가능성이 높다는 의미 사이에 매우 약한 긍정적 관계

### 이전 작업 캡슐화

In [ ]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [ ]:
bureau_agg_new = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg_new.head()

In [ ]:
bureau_agg.head()

값을 살펴보고 살펴보면 값이 같다는 것을 알게됩니다. 다른 데이터 프레임의 수치 통계를 계산하기 위해이 기능을 재사용 할 수 있습니다.

### Correlation Function

In [ ]:
def target_corrs(df):

    # List of correlations
    corrs = []

    # Iterate through the columns 
    for col in df.columns:
        print(col)
        # Skip the target column
        if col != 'TARGET':
            # Calculate correlation with the target
            corr = df['TARGET'].corr(df[col])

            # Append the list as a tuple
            corrs.append((col, corr))
            
    # Sort by absolute magnitude of correlations
    corrs = sorted(corrs, key = lambda x: abs(x[1]), reverse = True)
    
    return corrs

### Categorical Variables

각 범주 형 변수 내에서 각 범주의 값 카운트를 계산

In [ ]:
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
categorical.head()

In [ ]:
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])
categorical_grouped.head()

- sum : 연결된 클라이언트에 대한 해당 범주의 개수를 나타낸다
- mean : 정규화 된 개수를 나타낸다

In [ ]:
categorical_grouped.columns.levels[0][:10]

In [ ]:
categorical_grouped.columns.levels[1]

In [ ]:
group_var = 'SK_ID_CURR'

# Need to create new column names
columns = []

# Iterate through the variables names
for var in categorical_grouped.columns.levels[0]:
    # Skip the grouping variable
    if var != group_var:
        # Iterate through the stat names
        for stat in ['count', 'count_norm']:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (var, stat))

#  Rename the columns
categorical_grouped.columns = columns

categorical_grouped.head()

In [ ]:
train = train.merge(categorical_grouped, left_on = 'SK_ID_CURR', right_index = True, how = 'left')
train.head()

In [ ]:
train.shape

In [ ]:
train.iloc[:10, 123:]

### Function to Handle Categorical Variables

In [ ]:
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [ ]:
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

### 다른 데이터프레임에 연산 적용

- 다른 금융 기관과의 각 고객의 이전 대출에 대한 월별 정보 존재
- 이 데이터 프레임을 클라이언트 ID 인 SK_ID_CURR로 그룹화하는 대신 먼저 이전 대출의 ID 인 SK_ID_BUREAU로 데이터 프레임을 그룹화
- 각 대출에 대한 하나의 데이터 프레임 행을 제공
- SK_ID_CURR로 그룹화하고 각 클라이언트의 대출에 대한 집계를 계산
- 최종 결과는 각 고객에 대해 하나의 행이있는 데이터 프레임이되고 대출에 대한 통계가 계산

In [ ]:
# Read in bureau balance
bureau_balance = pd.read_csv("../input/home-credit-default-risk/bureau_balance.csv",encoding= 'unicode_escape')
bureau_balance.head()

각가의 대출에 대한 status 계산

In [ ]:
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

- MONTHS_BALANCE 열에는 "신청일에 대한 월 잔고"

In [ ]:
# Calculate value count statistics for each `SK_ID_CURR` 
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

각 대출에 대한 계산이 있습니다. 이제 각 클라이언트마다이를 집계해야합니다. 먼저 데이터 프레임을 병합하여이 작업을 수행 한 다음 모든 변수가 숫자이므로 통계를 다시 집계하면됩니다. 이번에는 SK_ID_CURR로 그룹화합니다.

In [ ]:
# Dataframe grouped by the loan
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

# Merge to include the SK_ID_CURR
bureau_by_loan = bureau_by_loan.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')

bureau_by_loan.head()

In [ ]:
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')
bureau_balance_by_client.head()

#### bureau_balance
- 각 대출별로 계산 된 통계 수치 그룹화
- 대출을 기준으로 각 범주 형 변수 그룹화의 가치 계산
- 통계와 융자 가치 합병
- 클라이언트 ID별로 결과 데이터 프레임 그룹화에 대한 계산 된 숫자 통계

In [ ]:
# Free up memory by deleting old objects
import gc
gc.enable()
del train, bureau, bureau_balance, bureau_agg, bureau_agg_new, bureau_balance_agg, bureau_balance_counts, bureau_by_loan, bureau_balance_by_client, bureau_counts
gc.collect()

In [ ]:
train = pd.read_csv("../input/home-credit-default-risk/application_train.csv",encoding= 'unicode_escape')
bureau = pd.read_csv("../input/home-credit-default-risk/bureau.csv",encoding= 'unicode_escape')
bureau_balance = pd.read_csv("../input/home-credit-default-risk/bureau_balance.csv",encoding= 'unicode_escape')

### Counts of Bureau Dataframe

In [ ]:
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

#### Aggregated Stats of Bureau Dataframe

In [ ]:
bureau_agg = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg.head()

### Value counts of Bureau Balance dataframe by loan

In [ ]:
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

### Bureau Balance 데이터 프레임을 대출로 집계

In [ ]:
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

#### client로 집계

In [ ]:
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

# Merge to include the SK_ID_CURR
bureau_by_loan = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left')

# Aggregate the stats for each client
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')

In [ ]:
original_features = list(train.columns)
print('Original Number of Features: ', len(original_features))

In [ ]:
train = train.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

# Merge with the monthly information grouped by client
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [ ]:
new_features = list(train.columns)
print('Number of features using previous loans from other institutions data: ', len(new_features))

### Missing Values

In [ ]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [ ]:
missing_train = missing_values_table(train)
missing_train.head(10)

결 측값이 90 %보다 큰 학습 또는 테스트 데이터의 열을 제거

In [ ]:
missing_train_vars = list(missing_train.index[missing_train['% of Total Values'] > 90])
len(missing_train_vars)

결 측값을 제거하기 전에 테스트 데이터에서 결 측값 백분율을 찾습니다. 그런 다음 학습 또는 테스트 데이터에서 결 측값이 90 %보다 큰 모든 열을 제거

테스트 데이터를 읽고 동일한 작업을 수행하고 테스트 데이터에서 누락 된 값을 살펴보기

모든 수와 집계 통계를 계산 했으므로 테스트 데이터를 적절한 데이터와 병합

### Calculate Information for Testing Data

In [ ]:
test = pd.read_csv("../input/home-credit-default-risk/application_test.csv",encoding= 'unicode_escape')

# Merge with the value counts of bureau
test = test.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
test = test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

# Merge with the value counts of bureau balance
test = test.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [ ]:
print('Shape of Testing Data: ', test.shape)

In [ ]:
train_labels = train['TARGET']

# Align the dataframes, this will remove the 'TARGET' column
train, test = train.align(test, join = 'inner', axis = 1)

train['TARGET'] = train_labels

In [ ]:
print('Training Data Shape: ', train.shape)
print('Testing Data Shape: ', test.shape)

- train data와 test data 프레임 동일한 열 갖게 정렬

In [ ]:
missing_test = missing_values_table(test)
missing_test.head(10)

In [ ]:
missing_test_vars = list(missing_test.index[missing_test['% of Total Values'] > 90])
len(missing_test_vars)

In [ ]:
missing_columns = list(set(missing_test_vars + missing_train_vars))
print('There are %d columns with more than 90%% missing in either the training or testing data.' % len(missing_columns))

In [ ]:
# Drop the missing columns
train = train.drop(columns = missing_columns)
test = test.drop(columns = missing_columns)

In [ ]:
train.to_csv('train_bureau_raw.csv', index = False)
test.to_csv('test_bureau_raw.csv', index = False)

### Correlations

target 과 변수 간의 상관관계

In [ ]:
corrs = train.corr()

In [ ]:
corrs = corrs.sort_values('TARGET', ascending = False)

# Ten most positive correlations
pd.DataFrame(corrs['TARGET'].head(10))

In [ ]:
pd.DataFrame(corrs['TARGET'].dropna().tail(10))

Target 과 상관계수가 높은 변수는 만든 변수, 그러나 상관계수가 높다 유용한 것은 아님 

### Collinear Variables

대상과 변수의 상관 관계뿐만 아니라 각 변수와 다른 모든 변수의 상관 관계도 계산할 수 있습니다. 이를 통해 데이터에서 제거해야 할 공선 변수가 있는지 확인할 수 있습니다.

In [ ]:
threshold = 0.8

# Empty dictionary to hold correlated variables
above_threshold_vars = {}

# For each column, record the variables that are above the threshold
for col in corrs:
    above_threshold_vars[col] = list(corrs.index[corrs[col] > threshold])

In [ ]:
cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []

# Iterate through columns and correlated columns
for key, value in above_threshold_vars.items():
    # Keep track of columns already examined
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            # Only want to remove one in a pair
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
            
cols_to_remove = list(set(cols_to_remove))
print('Number of columns to remove: ', len(cols_to_remove))

In [ ]:
train_corrs_removed = train.drop(columns = cols_to_remove)
test_corrs_removed = test.drop(columns = cols_to_remove)

print('Training Corrs Removed Shape: ', train_corrs_removed.shape)
print('Testing Corrs Removed Shape: ', test_corrs_removed.shape)

In [ ]:
train_corrs_removed.to_csv('train_bureau_corrs_removed.csv', index = False)
test_corrs_removed.to_csv('test_bureau_corrs_removed.csv', index = False)

### Modeling

In [ ]:
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import gc

import matplotlib.pyplot as plt

In [ ]:
def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [ ]:
def plot_feature_importances(df):
    """
    Plot importances returned by a model. This can work with any measure of
    feature importance provided that higher importance is better. 
    
    Args:
        df (dataframe): feature importances. Must have the features in a column
        called `features` and the importances in a column called `importance
        
    Returns:
        shows a plot of the 15 most importance features
        
        df (dataframe): feature importances sorted by importance (highest to lowest) 
        with a column for normalized importance
        """
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

### Control

In [ ]:
test_control  = pd.read_csv("../input/home-credit-default-risk/application_test.csv",encoding= 'unicode_escape')
train_control  = pd.read_csv("../input/home-credit-default-risk/application_train.csv",encoding= 'unicode_escape')

In [ ]:
submission, fi, metrics = model(train_control, test_control)